```
This notebook consist of code which is used to separate images into train and test folder. Also, we have a function which is used to randomly select n classes for n-class classification. Notebook is prepared as part of the semester project for CIS 731 ANN.

Submitted to : Prof C. K. Mohan

Submitted by : Laxman Kumar, Rima Shah

```

# Library Import

In [72]:
%matplotlib inline
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os,cv2
from sklearn.model_selection import train_test_split
import shutil

import torch
from torchvision import models
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn.functional as F
from collections import defaultdict
import torchvision

from PIL import Image, ImageDraw
import matplotlib.pyplot as plt

import os
import json
import re
import random
from math import sqrt
from tqdm import tqdm

import numpy as np
import pandas as pd
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

import matplotlib.pyplot as plt

import collections

### Separating the images into train and test folder based on the train and test text file.

Below code creates two new folder with train images and test images. It read the details of train images from text file and copy those images into the train folder. This process is repeated for test images also.

In [73]:
class_to_ix = {}
ix_to_class = {}
with open('food-101/meta/classes.txt', 'r') as txt:
    classes = [l.strip() for l in txt.readlines()]
    class_to_ix = dict(zip(classes, range(len(classes))))
    ix_to_class = dict(zip(range(len(classes)), classes))
    class_to_ix = {v: k for k, v in ix_to_class.items()}
sorted_class_to_ix = collections.OrderedDict(sorted(class_to_ix.items()))

In [96]:
def generateDataBasedOnNumberOfClass(src, dst,  symlinks = False, ignore = None,count=1,nclasses=3):
    if not os.path.exists(dst):
        os.makedirs(dst)
        shutil.copystat(src, dst)
    
    random.seed(100)
    lst = os.listdir(src)
    if(count==1):
        lst = random.sample(lst,nclasses)
    if ignore:
        excl = ignore(src, lst)
        lst = [x for x in lst if x not in excl]
    
        
    for item in lst:
        s = os.path.join(src, item)
        d = os.path.join(dst, item)
        
        if symlinks and os.path.islink(s):
            if os.path.lexists(d):
                os.remove(d)
            os.symlink(os.readlink(s), d)
            try:
                st = os.lstat(s)
                mode = stat.S_IMODE(st.st_mode)
                os.lchmod(d, mode)
            except:
                pass # lchmod not available
        elif os.path.isdir(s):
            generateDataBasedOnNumberOfClass(s, d, symlinks, ignore,count+1,nclasses)
        else:
            shutil.copy2(s, d)

def generate_dir_file_map(path):
    dir_files = defaultdict(list)
    with open(path, 'r') as txt:
        files = [l.strip() for l in txt.readlines()]
        for f in files:
            dir_name, id = f.split('/')
            dir_files[dir_name].append(id + '.jpg')
    return dir_files

train_dir_files = generate_dir_file_map('food-101/meta/train.txt')
test_dir_files = generate_dir_file_map('food-101/meta/test.txt')

def ignore_train(d, filenames):
    subdir = d.split('/')[-1]
    to_ignore = train_dir_files[subdir]
    return to_ignore

def ignore_test(d, filenames):
    #print(d)
    subdir = d.split('/')[-1]
    to_ignore = test_dir_files[subdir]
    return to_ignore

generateDataBasedOnNumberOfClass('food-101/images', './DataFolder/5Classes/test',
                                 ignore=ignore_train,count=1,nclasses=3)
generateDataBasedOnNumberOfClass('food-101/images', './DataFolder/5Classes/train',
                                 ignore=ignore_test,count=1,nclasses=3)

In [ ]:
def generateDataBasedOnNumberOfClass(n=3,train_source_path,
                                     test_source_path,
                                     train_dest_path,
                                     test_test_path):
    